# Description

**TODO**

# Modules

In [1]:
library(reticulate)
pd <- import("pandas")

# Settings

In [2]:
# reference panel
REFERENCE_PANEL <- "GTEX_V8"
# REFERENCE_PANEL = "1000G"

# prediction models
## mashr
EQTL_MODEL <- "MASHR"

# ## elastic net
# EQTL_MODEL = "ELASTIC_NET"

chromosome <- NULL

In [3]:
# Parameters
chromosome = 2
REFERENCE_PANEL = "GTEX_V8"
EQTL_MODEL = "MASHR"


In [4]:
paste0("Using reference panel: ", REFERENCE_PANEL)

[1] "Using reference panel: GTEX_V8"

In [5]:
paste0("Using eQTL model: ", EQTL_MODEL)

[1] "Using eQTL model: MASHR"

In [6]:
# chromosome must be provided as parameter
stopifnot(!is.null(chromosome))

# Paths

In [7]:
GENE_CORRS_DIR <- Sys.getenv("PHENOPLIER_PHENOMEXCAN_LD_BLOCKS_GENE_CORRS_DIR")
IRdisplay::display(GENE_CORRS_DIR)

[1] "/opt/data/data/phenomexcan/ld_blocks/gene_corrs"

In [8]:
INPUT_DIR <- file.path(GENE_CORRS_DIR, tolower(REFERENCE_PANEL), tolower(EQTL_MODEL), "by_chr")
IRdisplay::display(INPUT_DIR)

[1] "/opt/data/data/phenomexcan/ld_blocks/gene_corrs/gtex_v8/mashr/by_chr"

In [9]:
INPUT_FILE <- file.path(INPUT_DIR, paste0("gene_corrs-chr", chromosome, ".pkl"))
IRdisplay::display(INPUT_FILE)
stopifnot(file.exists(INPUT_FILE))

[1] "/opt/data/data/phenomexcan/ld_blocks/gene_corrs/gtex_v8/mashr/by_chr/gene_corrs-chr2.pkl"

In [10]:
OUTPUT_DIR <- file.path(INPUT_DIR, "corrected_positive_definite")
IRdisplay::display(OUTPUT_DIR)
dir.create(OUTPUT_DIR, recursive = TRUE, showWarnings = FALSE)

[1] "/opt/data/data/phenomexcan/ld_blocks/gene_corrs/gtex_v8/mashr/by_chr/corrected_positive_definite"

In [11]:
OUTPUT_FILE <- file.path(OUTPUT_DIR, paste0("gene_corrs-chr", chromosome, ".pkl"))
IRdisplay::display(OUTPUT_FILE)
if (file.exists(OUTPUT_FILE)) {
  IRdisplay::display("Output file exists, it will be overwritten")
}

[1] "/opt/data/data/phenomexcan/ld_blocks/gene_corrs/gtex_v8/mashr/by_chr/corrected_positive_definite/gene_corrs-chr2.pkl"

[1] "Output file exists, it will be overwritten"

# Functions

In [12]:
# taken from https://www.r-bloggers.com/2013/08/correcting-a-pseudo-correlation-matrix-to-be-positive-semidefinite/
# TODO: add documentation
CorrectCM <- function(CM, p = 0) {
  n <- dim(var(CM))[1L]
  E <- eigen(CM)
  CM1 <- E$vectors %*% tcrossprod(diag(pmax(E$values, p), n), E$vectors)
  Balance <- diag(1 / sqrt(diag(CM1)))
  CM2 <- Balance %*% CM1 %*% Balance
  return(CM2)
}

In [13]:
is_positive_definite <- function(data) {
  eigenvalues <- eigen(data)$values
  nonpositive_eigenvalues <- eigenvalues[eigenvalues <= 0]

  if (length(nonpositive_eigenvalues) > 0) {
    IRdisplay::display("We need to correct the data and make the matrix positive definite")
    return(FALSE)
  } else {
    IRdisplay::display("Matrix is already positive definite!")
    return(TRUE)
  }
}

# Load data

In [14]:
gene_corrs <- pd$read_pickle(INPUT_FILE)

In [15]:
dim(gene_corrs)

[1] 445 445

In [16]:
head(gene_corrs[1:10, 1:10])

,ENSG00000143727,ENSG00000115705,ENSG00000130508,ENSG00000182551,ENSG00000171865,ENSG00000171863,ENSG00000134326,ENSG00000134321,ENSG00000115738,ENSG00000134313
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
ENSG00000143727,1.00000000,-0.07840744,0.19581325,-0.031212187,0.034284308,-0.12374498,0.00000000,0.00000000,0,0
ENSG00000115705,-0.07840744,1.00000000,0.01839845,-0.048030022,-0.024985746,0.02461206,0.00000000,0.00000000,0,0
ENSG00000130508,0.19581325,0.01839845,1.00000000,0.178464442,0.071530193,-0.13140764,0.00000000,0.00000000,0,0
ENSG00000182551,-0.03121219,-0.04803002,0.17846444,1.000000000,-0.009727435,-0.09785083,-0.02614311,-0.06780080,0,0
ENSG00000171865,0.03428431,-0.02498575,0.07153019,-0.009727435,1.000000000,-0.06822706,0.01867079,0.04843000,0,0
ENSG00000171863,-0.12374498,0.02461206,-0.13140764,-0.097850834,-0.068227057,1.00000000,-0.07223635,-0.07550064,0,0


# Check positive definiteness

In [17]:
is_positive_definite(gene_corrs)

[1] "We need to correct the data and make the matrix positive definite"

[1] FALSE

In [18]:
# see eigenvalues
eigenvalues <- eigen(gene_corrs)$values

In [19]:
nonpositive_eigenvalues <- eigenvalues[eigenvalues <= 0]
IRdisplay::display(length(nonpositive_eigenvalues))
IRdisplay::display(nonpositive_eigenvalues)

[1] 22

[1] -0.007879462 -0.075904169 -0.089130062 -0.093904599 -0.103049786
 [6] -0.117544357 -0.152926544 -0.156526537 -0.250051757 -0.268007608
[11] -0.286957250 -0.359292159 -0.391492308 -0.473873246 -0.488917558
[16] -0.577738620 -0.592093912 -0.788551122 -0.812294577 -0.875576360
[21] -1.051005781 -1.660436783

In [20]:
if (length(eigenvalues[eigenvalues <= 0]) == 0) { quit() }

# Make matrix positive definite if needed

In [21]:
gene_corrs_corrected <- CorrectCM(gene_corrs, 1)

In [22]:
dimnames(gene_corrs_corrected)[[1]] <- rownames(gene_corrs)

In [23]:
dimnames(gene_corrs_corrected)[[2]] <- colnames(gene_corrs)

In [24]:
gene_corrs_corrected <- as.data.frame(gene_corrs_corrected)

In [25]:
dim(gene_corrs_corrected)

[1] 445 445

# Check positive definiteness of corrected matrix

In [26]:
is_positive_definite(gene_corrs_corrected)

[1] "Matrix is already positive definite!"

[1] TRUE

In [27]:
# see eigenvalues
eigenvalues <- eigen(gene_corrs_corrected)$values

In [28]:
nonpositive_eigenvalues <- eigenvalues[eigenvalues <= 0]
IRdisplay::display(length(nonpositive_eigenvalues))
IRdisplay::display(nonpositive_eigenvalues)

[1] 0

numeric(0)

In [29]:
stopifnot(length(eigenvalues[eigenvalues <= 0]) == 0)

In [30]:
# quick and visual comparison of the two matrices
IRdisplay::display(head(gene_corrs[1:10, 1:10]))
IRdisplay::display(head(gene_corrs_corrected[1:10, 1:10]))

,ENSG00000143727,ENSG00000115705,ENSG00000130508,ENSG00000182551,ENSG00000171865,ENSG00000171863,ENSG00000134326,ENSG00000134321,ENSG00000115738,ENSG00000134313
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
ENSG00000143727,1.00000000,-0.07840744,0.19581325,-0.031212187,0.034284308,-0.12374498,0.00000000,0.00000000,0,0
ENSG00000115705,-0.07840744,1.00000000,0.01839845,-0.048030022,-0.024985746,0.02461206,0.00000000,0.00000000,0,0
ENSG00000130508,0.19581325,0.01839845,1.00000000,0.178464442,0.071530193,-0.13140764,0.00000000,0.00000000,0,0
ENSG00000182551,-0.03121219,-0.04803002,0.17846444,1.000000000,-0.009727435,-0.09785083,-0.02614311,-0.06780080,0,0
ENSG00000171865,0.03428431,-0.02498575,0.07153019,-0.009727435,1.000000000,-0.06822706,0.01867079,0.04843000,0,0
ENSG00000171863,-0.12374498,0.02461206,-0.13140764,-0.097850834,-0.068227057,1.00000000,-0.07223635,-0.07550064,0,0


,ENSG00000143727,ENSG00000115705,ENSG00000130508,ENSG00000182551,ENSG00000171865,ENSG00000171863,ENSG00000134326,ENSG00000134321,ENSG00000115738,ENSG00000134313
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
ENSG00000143727,1.00000000,-0.03471638,0.09231575,0.04634517,0.04685952,-0.08016655,0.0072143967,8.559309e-03,0.0012442669,-2.902927e-03
ENSG00000115705,-0.03471638,1.00000000,-0.02941794,-0.01190866,-0.01687068,0.02580236,0.0009909984,4.706685e-04,0.0003051304,7.024305e-05
ENSG00000130508,0.09231575,-0.02941794,1.00000000,0.08274796,0.04527668,-0.09825233,0.0020353241,-5.412952e-05,0.0001379938,-9.027237e-04
ENSG00000182551,0.04634517,-0.01190866,0.08274796,1.00000000,0.01438333,-0.05545717,-0.0246278726,-3.029527e-02,-0.0014908876,-3.005412e-03
ENSG00000171865,0.04685952,-0.01687068,0.04527668,0.01438333,1.00000000,-0.04665396,0.0255013948,2.771904e-02,0.0013748303,1.021597e-03
ENSG00000171863,-0.08016655,0.02580236,-0.09825233,-0.05545717,-0.04665396,1.00000000,-0.0446797025,-4.048404e-02,0.0001385210,-7.872039e-03


Both matrices should "look" similar. We are not interested in perfectly accurate correlation values (they are already inaccurate).

# Save

In [31]:
py_save_object(gene_corrs_corrected, OUTPUT_FILE)